In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [3]:
# 1. Encoded pixels to masks and vice versa functions
def encoded_pixels_to_masks(fname: str, df: pd.DataFrame):
    fname_df = df[df['ImageId'] == fname]
    masks = np.zeros((256 * 1600, 4), dtype=int)

    for i_row, row in fname_df.iterrows():
        cls_id = row['ClassId']
        encoded_pixels = row['EncodedPixels']
        if encoded_pixels is not np.nan:
            pixel_list = list(map(int, encoded_pixels.split(' ')))
            for i in range(0, len(pixel_list), 2):
                start_pixel = pixel_list[i] - 1
                num_pixel = pixel_list[i+1]
                masks[start_pixel:(start_pixel+num_pixel), cls_id-1] = 1

    masks = masks.reshape(256, 1600, 4, order='F')
    return masks

In [4]:
# 2. Dataset class
class SeverstalSteelDataset(Dataset):
    def __init__(self, fnames, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.fnames = fnames
        self.transform = transform

    def __len__(self):
        return len(self.fnames)

    def __getitem__(self, idx):
        img_id = self.fnames[idx]
        img_path = os.path.join(self.img_dir, img_id)
        img = np.array(Image.open(img_path).convert('RGB'))  # Загружаем изображение
        masks = encoded_pixels_to_masks(img_id, self.df)  # Генерируем маску
        if self.transform:
            img = self.transform(image=img)['image']
        img = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1)  # Преобразуем в тензор
        masks = torch.tensor(masks, dtype=torch.float32).permute(2, 0, 1)  # Преобразуем в тензор
        return img_id, img, masks

In [5]:
# 3. Optional collate function
def collate_fn(batch_items):
    batched_fnames = [item[0] for item in batch_items]
    batched_imgs = torch.stack([item[1] for item in batch_items])
    batched_masks = torch.stack([item[2] for item in batch_items])
    return batched_fnames, batched_imgs, batched_masks

In [6]:
# 4. Model class
!pip install segmentation-models-pytorch
import segmentation_models_pytorch as smp

class SegModel(torch.nn.Module):
    def __init__(self):
        super(SegModel, self).__init__()
        self.model = smp.Unet(encoder_name='resnet34', encoder_weights='imagenet', classes=4, activation=None)

    def forward(self, x):
        return self.model(x)

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 59.0 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16425 sha256=017d8566d21e456d109e4cbbb9b2411f91684a0d0d28228b397b5f32b6367862
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=d10b06c96a872c2e0d0c536b40f72104553996c451078947c7df5797722922f1
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorc

In [7]:
# 5. Data loading function
def load_data(train_val_csv, test_csv, train_val_img_dir, test_img_dir):
    train_val_df = pd.read_csv(train_val_csv)
    train_val_fnames = pd.unique(train_val_df.ImageId)

    test_df = pd.read_csv(test_csv)
    test_fnames = pd.unique(test_df.ImageId)

    # Split train and val data
    train_fnames, val_fnames = train_test_split(train_val_fnames, test_size=0.2, random_state=42)

    train_dataset = SeverstalSteelDataset(train_fnames, train_val_df, train_val_img_dir)
    val_dataset = SeverstalSteelDataset(val_fnames, train_val_df, train_val_img_dir)
    test_dataset = SeverstalSteelDataset(test_fnames, test_df, test_img_dir)

    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn, num_workers=4)

    return train_loader, val_loader, test_loader

In [8]:
# 6. Model initialization
def init_model():
    model = SegModel()
    criterion = torch.nn.BCEWithLogitsLoss()  # Используем BCE с логитами для многоклассовой задачи
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    return model, criterion, optimizer

In [9]:
# 7. Dice Score function
def dice_score(preds, targets, smooth=1e-6):
    preds = preds.reshape(-1)
    targets = targets.reshape(-1)

    intersection = (preds * targets).sum()  # Считаем пересечение
    return (2. * intersection + smooth) / (preds.sum() + targets.sum() + smooth)  # Добавляем небольшое число для избежания деления на 0

In [10]:
# 8. Dice Score per class
def dice_score_per_class(preds, targets, smooth=1e-6):
    dice_per_class = []
    for i in range(preds.shape[1]):
        dice = dice_score(preds[:, i], targets[:, i], smooth)
        dice_per_class.append(dice.item())
    return dice_per_class

In [11]:
# 9. Average Dice Score
def average_dice_score(preds, targets, smooth=1e-6):
    dice_per_class = dice_score_per_class(preds, targets, smooth)
    mean_dice = np.mean(dice_per_class)
    return mean_dice, dice_per_class

In [12]:
# Задание индивидуальных порогов для каждого класса
thresholds = [0.5, 0.5, 0.4, 0.5]

# Модификация функции для использования индивидуальных порогов
def masks_to_encoded_pixels(mask, class_idx, thresholds):
    threshold = thresholds[class_idx]  # Берем порог для текущего класса
    pixels = mask.flatten(order='F')
    pixels = np.where(pixels > threshold, 1, 0)  # Применяем индивидуальный порог
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [13]:
# 10. Training function
def train(model, criterion, optimizer, loader, epochs, device):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for batch_fnames, batch_imgs, batch_masks in loader:
            batch_imgs, batch_masks = batch_imgs.to(device), batch_masks.to(device)

            optimizer.zero_grad()
            outputs = model(batch_imgs)
            loss = criterion(outputs, batch_masks)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(loader)}")
    return model

In [14]:
# 11. Validation function
def validate(model, val_loader, device):
    model.eval()
    dice_scores = []
    with torch.no_grad():
        for batch_fnames, batch_imgs, batch_masks in val_loader:
            batch_imgs, batch_masks = batch_imgs.to(device), batch_masks.to(device)
            outputs = model(batch_imgs)
            preds = torch.sigmoid(outputs) > 0.5  # Применяем порог для получения масок
            preds = preds.cpu().numpy()
            batch_masks = batch_masks.cpu().numpy()

            for i in range(len(batch_fnames)):
                for cls_id in range(4):
                    dice = dice_score(preds[i, cls_id], batch_masks[i, cls_id])
                    dice_scores.append(dice)

    mean_dice = np.mean(dice_scores)
    print(f"Average Dice Score: {mean_dice:.4f}")
    for cls_id in range(4):
        cls_dice = np.mean([score for i, score in enumerate(dice_scores) if i % 4 == cls_id])
        print(f"Dice Score for Class {cls_id + 1}: {cls_dice:.4f}")

In [15]:
# Генерация submission с индивидуальными порогами
def generate_submission(model, test_loader, device, thresholds):
    model.eval()
    submission = []

    with torch.no_grad():
        for batch_fnames, batch_imgs, _ in test_loader:
            batch_imgs = batch_imgs.to(device)
            outputs = model(batch_imgs)
            preds = torch.sigmoid(outputs)  # Применяем сигмоиду

            preds = preds.cpu().numpy()
            for i, fname in enumerate(batch_fnames):
                encoded_pixels_list = []
                for cls_id in range(4):  # Проходим по каждому классу
                    encoded_pixels = masks_to_encoded_pixels(preds[i, cls_id], cls_id, thresholds)
                    encoded_pixels_list.append((fname, cls_id + 1, encoded_pixels))

                submission.extend(encoded_pixels_list)

    # Создаем DataFrame для submission
    submission_df = pd.DataFrame(submission, columns=['ImageId', 'ClassId', 'EncodedPixels'])
    submission_df.to_csv('/content/gdrive/MyDrive/ColabNotebooks/2024/HW2/submission.csv', index=False)

In [16]:
# 12. Main execution block
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the data
train_loader, val_loader, test_loader = load_data(
    '/content/gdrive/MyDrive/ColabNotebooks/2024/HW2/train.csv',
    '/content/gdrive/MyDrive/ColabNotebooks/2024/HW2/sample_submission.csv',
    '/content/gdrive/MyDrive/ColabNotebooks/2024/HW2/train_images/',
    '/content/gdrive/MyDrive/ColabNotebooks/2024/HW2/test_images/'
)

# Initialize the model
model, criterion, optimizer = init_model()
model.to(device)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 187MB/s]


SegModel(
  (model): Unet(
    (encoder): ResNetEncoder(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchN

In [17]:
# Train the model
train(model, criterion, optimizer, train_loader, epochs=2, device=device)

Epoch 1/2, Loss: 0.24148171116923284
Epoch 2/2, Loss: 0.0609162078201324


SegModel(
  (model): Unet(
    (encoder): ResNetEncoder(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchN

In [18]:
# Validate the model
validate(model, val_loader, device)

Average Dice Score: 0.8026
Dice Score for Class 1: 0.7444
Dice Score for Class 2: 0.9753
Dice Score for Class 3: 0.5915
Dice Score for Class 4: 0.8993


In [19]:
# Предсказания на тестовом наборе и генерация submission файла
generate_submission(model, test_loader, device, thresholds)